In [0]:
import pyspark.sql.functions as F

In [0]:
df = spark.range(0, 150000000, 1, 32) \
            .select(
                "id", 
                F.round(F.rand() * 1000, 2).alias("amount"),
                (F.col("id") % 10).alias("country_id"),
                (F.col("id") % 100).alias("store_id")
            )

In [0]:
df.display()

In [0]:
df.write.mode("overwrite").saveAsTable("guns.bronze.150_billion")

In [0]:
store_df = spark.range(0, 99) \
            .select(
                "id",
                F.round(F.rand() * 100, 0).alias("employees"),
                (F.col("id") % 10).alias("country_id"),
                F.expr("uuid()").alias("name")
            )
store_df.display()

In [0]:
store_df.write.mode("overwrite").saveAsTable("guns.bronze.store_100")

In [0]:
countries = [
    (0, "India"),
    (1, "Canada"),
    (2, "Mexico"),
    (3, "China"),
    (4, "Germany"),
    (5, "UK"),
    (6, "Japan"),
    (7, "Korea"),
    (8, "Austrialia"),
    (9, "France"),
    (10, "Spain"),
    (11, "USA"),
] 
cols = ["id", "name"]
country_df = spark.createDataFrame(data=countries, schema=cols)
country_df.display()

In [0]:
country_df.write.mode("overwrite").saveAsTable("guns.bronze.country_11")

In [0]:
joined_df = spark.sql("""
                      SELECT 
                        t.id,
                        t.amount,
                        c.name,
                        s.employees,
                        s.name
                      FROM guns.bronze.150_billion t
                      LEFT JOIN guns.bronze.store_100 s
                      ON s.id = t.store_id
                      LEFT JOIN guns.bronze.country_11 c
                      ON t.country_id = c.id
                      """)

In [0]:
joined_df.display()

In [0]:
%sql
SELECT 
    country_id,
    COUNT(*),
    AVG(amount)
FROM guns.bronze.`150_billion`
GROUP BY country_id